In [1]:
# Set path variables
import os
import sys

cwd = os.getcwd()
project_dir = os.path.abspath(os.path.join(cwd, os.pardir))
sys.path.append(project_dir)
data_path = os.path.join(project_dir, 'data/')
print(project_dir)
print(data_path)

/home/abys/projects/tagging-music-sequences
/home/abys/projects/tagging-music-sequences/data/


In [2]:
# for data loading process
from src.data_loader import *
from torch.utils.data import DataLoader
import pandas as pd

import math
import numpy as np

import torch
from torch import nn
import torch.optim as optim
from torch.autograd import Variable
import os
from tqdm import tqdm

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [4]:
# Make sure CUDA is available
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(torch.cuda.current_device())
    print(f"Current CUDA Device Name: {device_name}")
else:
    print("CUDA is not available. Please check your system's configuration.")

Current CUDA Device Name: NVIDIA A30


# CNN+Attention

## Data loading

In [5]:
# Load label annotation csv
train_annotations = pd.read_csv(data_path + 'mtat_train_label.csv', index_col=0).reset_index(drop=True)
val_annotations = pd.read_csv(data_path + 'mtat_val_label.csv', index_col=0).reset_index(drop=True)
test_annotations = pd.read_csv(data_path + 'mtat_test_label.csv', index_col=0).reset_index(drop=True)

### FOR RAW AUDIO DATA

Set transformation parameter to None

In [6]:
# Define global parameters across all classes
DATA_DIR = data_path
SAMPLE_RATE = 16000
DURATION_IN_SEC = 15

train_data = AudioDS(annotations_file=train_annotations, 
                     data_dir=DATA_DIR, 
                     target_sample_rate=SAMPLE_RATE, 
                     target_length=DURATION_IN_SEC, 
                     transformation=None)

val_data = AudioDS(annotations_file=val_annotations,
                     data_dir=DATA_DIR,
                     target_sample_rate=SAMPLE_RATE,
                     target_length=DURATION_IN_SEC,
                     transformation=None)

test_data = AudioDS(annotations_file=val_annotations,
                     data_dir=DATA_DIR,
                     target_sample_rate=SAMPLE_RATE,
                     target_length=DURATION_IN_SEC,
                     transformation=None)

In [7]:
# Load data from created datasets
BATCH_SIZE = 8

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [8]:
# Display batch information
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([8, 1, 240000])
Labels batch shape: torch.Size([8, 50])


In [9]:
# Retrieve a sample
idx = 0
waveform = train_features[idx]
label = train_labels[idx]
decoded_labels = train_data.decode_labels(label)
file_path = train_data.get_filepath(idx)

print(f"Audio file path: {file_path}")
print(f"Label: {label}")
print(f"Decoded labels: {decoded_labels}")

Audio file path: /home/abys/projects/tagging-music-sequences/data/mtat/0/paul_berget-the_siena_manuscript_on_steel_string_guitar-06-recercar_6_steel_string_guitar-30-59.mp3
Label: tensor([False, False, False, False, False, False, False,  True, False, False,
        False,  True, False, False, False, False, False, False, False,  True,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False])
Decoded labels: ['drum', 'male', 'singer']


In [10]:
# shape of waveform
# first element: number of channels in our case 1
# second element: number of samples in 30 seconds audio at a sampling rate of 16000 samples/s 
# -> 480000 = 30s * 16000 samples/s
waveform.shape

torch.Size([1, 240000])

## Build Model

### Front-end CNN+waveform

In [12]:
class Conv1(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(Conv1, self).__init__()
        self.conv = nn.Conv1d(input_channels, output_channels, 1)
        self.bn = nn.BatchNorm1d(output_channels)
        self.relu = nn.ReLU()
    def forward(self, x):
        out = self.relu(self.bn(self.conv(x)))
        return out


class Conv7(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(Conv7, self).__init__()
        self.conv = nn.Conv1d(input_channels, output_channels, 7, padding=3)
        self.bn = nn.BatchNorm1d(output_channels)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.relu(self.bn(self.conv(x)))
        return out


class Conv3(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(Conv3, self).__init__()
        self.conv = nn.Conv1d(input_channels, output_channels, 3, padding=1)
        self.bn = nn.BatchNorm1d(output_channels)
        self.relu = nn.ReLU()
        self.mp = nn.MaxPool1d(3)

    def forward(self, x):
        out = self.mp(self.relu(self.bn(self.conv(x))))
        return out


class InitConv(nn.Module):
    def __init__(self, output_channels):
        super(InitConv, self).__init__()
        self.conv = nn.Conv1d(1, output_channels, 3, stride=3, padding=1)
        self.bn = nn.BatchNorm1d(output_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv(x)
        out = self.relu(self.bn(out))
        return out

In [13]:
class LeeConvModule(nn.Module):
    def __init__(self, conv_channels, num_classes):
        super(LeeConvModule, self).__init__()
        
        # initial convolution
        self.init_conv = InitConv(conv_channels)

        # stack convolution
        c = conv_channels
        channels = [c, c, c, c*2, c*2, c*2, c*2, c*2, c*4]
        self.convs = nn.ModuleList([Conv3(channels[i], channels[i+1]) for i in range(len(channels)-1)])

        # Adding three Conv7 layers
        self.conv7x1_1 = Conv7(channels[-1], channels[-1])
        
        # Final classification layer
        self.classifier = nn.Linear(channels[-1], num_classes)

    def forward(self, x):
        out = self.init_conv(x)
        for layer in self.convs:
            out = layer(out)
            
#         # Pass through Conv7 layers
#         out = self.conv7x1_1(out)
                                     
        # Global average pooling before the classification layer
        out = torch.mean(out, dim=-1)

        # Apply the final classifier
        logits = self.classifier(out)
        return logits

In [14]:
class TrainLeeConvModule:
    def __init__(self, conv_channel, num_classes=50, lr=0.001, epochs=10, model_save_path='../models/model.pth', use_cuda=True):
        self.lr = lr
        self.epochs = epochs
        self.model_save_path = model_save_path
        self.use_cuda = use_cuda and torch.cuda.is_available()
        
        # Initialize the model
        self.model = LeeConvModule(conv_channel, num_classes)
        if self.use_cuda:
            self.model.cuda()

        # Loss function and optimizer
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)

    def save_checkpoint(self, epoch):
        checkpoint_path = f'{self.model_save_path}_epoch_{epoch}.pth'
        torch.save(self.model.state_dict(), checkpoint_path)
        print(f'Model saved to {checkpoint_path}')

    def train(self, train_features, train_labels):
        num_batches = train_features.shape[0]
        # Iterate over batches with tqdm for progress display
        tqdm_bar = tqdm(range(self.epochs), desc=f'Training Progress', leave=True)
        for epoch in tqdm_bar:
            self.model.train()
            running_loss = 0.0

            if self.use_cuda:
                    train_features, train_labels = train_features.cuda(), train_labels.cuda()

            # Zero the parameter gradients
            self.optimizer.zero_grad()

            # Forward pass
            outputs = self.model(train_features)
            loss = self.criterion(outputs, train_labels)

            # Backward pass and optimize
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()
            avg_loss = running_loss / len(train_features)
            # Update tqdm bar
            tqdm_bar.set_postfix(loss=avg_loss)

        
            # Save model checkpoint
            self.save_checkpoint(epoch+1)

            # Print average loss at the end of the epoch
            avg_loss = running_loss / num_batches
            print(f'Epoch [{epoch+1}/{self.epochs}] completed, Average Loss: {avg_loss:.4f}')

        # Save the final model
        torch.save(self.model.state_dict(), self.model_save_path)
        print(f'Final model saved to {self.model_save_path}')

In [15]:

# Initialize training
config = {
    'conv_channel':64,
    'num_classes': 50,   # As per your requirement
    'lr': 0.001,         # Learning rate
    'epochs': 10,        # Number of epochs
    'model_save_path': '../models/wave_conv_model.pth', # Path to save the model
}

trainer = TrainLeeConvModule(**config)
trainer.train(train_features, train_labels)


Training Progress:   0%|          | 0/10 [00:12<?, ?it/s]


RuntimeError: Found dtype Bool but expected Float

### different architecture

In [20]:
import torch.nn.functional as F

class WaveformNet(nn.Module):
    def __init__(self, num_classes=50):
        super(WaveformNet, self).__init__()
        
        # Strided convolution to reduce dimensionality
        self.strided_conv = nn.Conv1d(1, 128, kernel_size=3, stride=3, padding=1)
        self.bn0 = nn.BatchNorm1d(128)

        # Convolutional blocks
        self.conv_blocks = nn.ModuleList()
        in_channels = 128
        for i in range(9):
            out_channels = 128  # Keeping the channel size consistent
            self.conv_blocks.append(nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1))
            self.conv_blocks.append(nn.BatchNorm1d(out_channels))
            self.conv_blocks.append(nn.ReLU())
            self.conv_blocks.append(nn.MaxPool1d(kernel_size=3, stride=3))
            in_channels = out_channels  # This will be 128 throughout as out_channels is constant

        # Global max pooling
        self.global_max_pool = nn.AdaptiveMaxPool1d(1)

        # Fully connected layers
        # Adjust the input size of the first FC layer based on the output of the last convolutional block
        self.fc1 = nn.Linear(128, 256)  # Adjust 128 based on the output channels of the last conv block
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        # Initial strided convolution
        x = F.relu(self.bn0(self.strided_conv(x)))

        # Convolutional blocks
        for block in self.conv_blocks:
            x = block(x)
        
        # Global max pooling
        x = self.global_max_pool(x)
        x = x.view(x.size(0), -1)  # Flatten

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [25]:
from tqdm import tqdm
import torch.optim as optim

class TrainWaveformNet:
    def __init__(self, num_classes, learning_rate, epochs, model_save_path, use_cuda):
        self.epochs = epochs
        self.model_save_path = model_save_path
        self.use_cuda = use_cuda and torch.cuda.is_available()
        self.device = torch.device("cuda" if self.use_cuda else "cpu")
        
        # Initialize the model
        self.model = WaveformNet(num_classes=num_classes)
        self.model.to(self.device)

        # Loss function and optimizer
        self.criterion = nn.BCEWithLogitsLoss()  # Using BCEWithLogitsLoss for stability
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    def save_checkpoint(self, epoch):
        # Save a checkpoint of the model
        checkpoint_path = f'{self.model_save_path}_checkpoint_epoch_{epoch}.pth'
        torch.save({
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
        }, checkpoint_path)
        print(f'Checkpoint saved: {checkpoint_path}')

    def train(self, train_features, train_labels):
        train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

        pbar = tqdm(total=self.epochs * len(train_loader), desc='Training', leave=True)

        for epoch in range(self.epochs):
            self.model.train()
            running_loss = 0.0

            for batch_idx, (batch_features, batch_labels) in enumerate(train_loader):
                # Convert batch_labels to float
                batch_labels = batch_labels.float()

                batch_features, batch_labels = batch_features.to(self.device), batch_labels.to(self.device)

                # Forward pass
                self.optimizer.zero_grad()
                outputs = self.model(batch_features)
                loss = self.criterion(outputs, batch_labels)
                running_loss += loss.item()

                # Backward and optimize
                loss.backward()
                self.optimizer.step()

                # Update progress bar
                pbar.update(1)

            # Update progress bar with epoch and batch information
            pbar.set_description(f'Epoch [{epoch+1}/{self.epochs}], Batch [{batch_idx+1}/{len(train_loader)}]')
            pbar.set_postfix(loss=running_loss / (batch_idx + 1))
            pbar.update(1)

            # Save checkpoint after each epoch
            self.save_checkpoint(epoch)
        
        
        # Save the final model
        final_model_path = f'{self.model_save_path}_final.pth'
        torch.save(self.model.state_dict(), final_model_path)
        print(f'Final model saved to {final_model_path}')

        # Close the progress bar
        pbar.close()

In [26]:
# Parameters for the model and training
num_classes = 50
learning_rate = 0.001
epochs = 2
model_save_path = '../models/waveform_model'  # Adjust the path as needed

# Assume train_features and train_labels are already tensors with the correct shape
# Example usage:
trainer = TrainWaveformNet(num_classes, learning_rate, epochs, model_save_path, use_cuda=torch.cuda.is_available())
trainer.train(train_features, train_labels)  # train_features and train_labels should be tensors





Training:   0%|          | 0/3246 [00:00<?, ?it/s]


Training:   0%|          | 1/3246 [00:00<10:07,  5.34it/s]


Training:   0%|          | 2/3246 [00:00<08:38,  6.26it/s]


Training:   0%|          | 3/3246 [00:00<13:03,  4.14it/s]


Training:   0%|          | 4/3246 [00:00<10:52,  4.97it/s]


Training:   0%|          | 5/3246 [00:00<09:37,  5.61it/s]


Training:   0%|          | 6/3246 [00:01<08:51,  6.10it/s]


Training:   0%|          | 7/3246 [00:01<08:18,  6.50it/s]


Training:   0%|          | 8/3246 [00:01<07:58,  6.77it/s]


Training:   0%|          | 9/3246 [00:01<07:44,  6.97it/s]


Training:   0%|          | 10/3246 [00:01<07:34,  7.12it/s]


Training:   0%|          | 11/3246 [00:01<07:27,  7.24it/s]


Training:   0%|          | 12/3246 [00:01<07:22,  7.31it/s]


Training:   0%|          | 13/3246 [00:02<07:19,  7.36it/s]


Training:   0%|          | 14/3246 [00:02<07:17,  7.39it/s]


Training:   0%|          | 15/3246 [00:02<07:15,  7.43it/s]


Training:   0%|        

Training:   4%|▎         | 115/3246 [00:19<08:27,  6.17it/s]


Training:   4%|▎         | 116/3246 [00:19<08:10,  6.39it/s]


Training:   4%|▎         | 117/3246 [00:20<08:00,  6.51it/s]


Training:   4%|▎         | 118/3246 [00:21<20:15,  2.57it/s]


Training:   4%|▎         | 119/3246 [00:21<16:17,  3.20it/s]


Training:   4%|▎         | 120/3246 [00:21<13:33,  3.84it/s]


Training:   4%|▎         | 121/3246 [00:21<11:40,  4.46it/s]


Training:   4%|▍         | 122/3246 [00:21<10:22,  5.02it/s]


Training:   4%|▍         | 123/3246 [00:21<09:29,  5.49it/s]


Training:   4%|▍         | 124/3246 [00:21<08:53,  5.86it/s]


Training:   4%|▍         | 125/3246 [00:22<08:27,  6.15it/s]


Training:   4%|▍         | 126/3246 [00:22<08:10,  6.36it/s]


Training:   4%|▍         | 127/3246 [00:22<11:08,  4.67it/s]


Training:   4%|▍         | 128/3246 [00:22<09:54,  5.25it/s]


Training:   4%|▍         | 129/3246 [00:22<09:05,  5.72it/s]


Training:   4%|▍         | 130/3246 [00:22<08:28,  6.12

Training:   7%|▋         | 228/3246 [00:39<08:49,  5.70it/s]


Training:   7%|▋         | 229/3246 [00:39<08:18,  6.05it/s]


Training:   7%|▋         | 230/3246 [00:39<07:57,  6.31it/s]


Training:   7%|▋         | 231/3246 [00:39<07:45,  6.48it/s]


Training:   7%|▋         | 232/3246 [00:39<07:38,  6.58it/s]


Training:   7%|▋         | 233/3246 [00:40<16:33,  3.03it/s]


Training:   7%|▋         | 234/3246 [00:40<13:36,  3.69it/s]


Training:   7%|▋         | 235/3246 [00:40<11:34,  4.33it/s]


Training:   7%|▋         | 236/3246 [00:40<10:08,  4.95it/s]


Training:   7%|▋         | 237/3246 [00:41<09:06,  5.51it/s]


Training:   7%|▋         | 238/3246 [00:41<08:23,  5.98it/s]


Training:   7%|▋         | 239/3246 [00:41<07:52,  6.36it/s]


Training:   7%|▋         | 240/3246 [00:41<07:31,  6.66it/s]


Training:   7%|▋         | 241/3246 [00:41<07:18,  6.85it/s]


Training:   7%|▋         | 242/3246 [00:41<07:07,  7.03it/s]


Training:   7%|▋         | 243/3246 [00:41<06:59,  7.16

Training:  11%|█         | 341/3246 [00:56<06:32,  7.41it/s]


Training:  11%|█         | 342/3246 [00:56<09:27,  5.12it/s]


Training:  11%|█         | 343/3246 [00:56<08:35,  5.63it/s]


Training:  11%|█         | 344/3246 [00:56<07:57,  6.08it/s]


Training:  11%|█         | 345/3246 [00:57<07:30,  6.44it/s]


Training:  11%|█         | 346/3246 [00:57<07:12,  6.70it/s]


Training:  11%|█         | 347/3246 [00:57<07:00,  6.90it/s]


Training:  11%|█         | 348/3246 [00:57<06:50,  7.06it/s]


Training:  11%|█         | 349/3246 [00:57<06:44,  7.16it/s]


Training:  11%|█         | 350/3246 [00:57<06:41,  7.22it/s]


Training:  11%|█         | 351/3246 [00:57<06:37,  7.28it/s]


Training:  11%|█         | 352/3246 [00:57<06:34,  7.33it/s]


Training:  11%|█         | 353/3246 [00:58<06:33,  7.35it/s]


Training:  11%|█         | 354/3246 [00:58<06:33,  7.36it/s]


Training:  11%|█         | 355/3246 [00:58<09:26,  5.10it/s]


Training:  11%|█         | 356/3246 [00:58<08:32,  5.64

Training:  14%|█▍        | 454/3246 [01:13<06:40,  6.97it/s]


Training:  14%|█▍        | 455/3246 [01:13<06:34,  7.08it/s]


Training:  14%|█▍        | 456/3246 [01:13<06:28,  7.19it/s]


Training:  14%|█▍        | 457/3246 [01:13<06:23,  7.26it/s]


Training:  14%|█▍        | 458/3246 [01:13<06:21,  7.30it/s]


Training:  14%|█▍        | 459/3246 [01:13<06:19,  7.35it/s]


Training:  14%|█▍        | 460/3246 [01:14<06:17,  7.39it/s]


Training:  14%|█▍        | 461/3246 [01:14<06:14,  7.44it/s]


Training:  14%|█▍        | 462/3246 [01:14<09:01,  5.14it/s]


Training:  14%|█▍        | 463/3246 [01:14<08:11,  5.66it/s]


Training:  14%|█▍        | 464/3246 [01:14<07:36,  6.09it/s]


Training:  14%|█▍        | 465/3246 [01:14<07:11,  6.44it/s]


Training:  14%|█▍        | 466/3246 [01:15<06:54,  6.71it/s]


Training:  14%|█▍        | 467/3246 [01:15<06:42,  6.91it/s]


Training:  14%|█▍        | 468/3246 [01:15<06:33,  7.06it/s]


Training:  14%|█▍        | 469/3246 [01:15<06:27,  7.17

...

Epoch [1/2], Batch [1623/1623]:  92%|█████████▏| 2994/3246 [07:35<00:38,  6.52it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  92%|█████████▏| 2995/3246 [07:35<00:37,  6.70it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  92%|█████████▏| 2996/3246 [07:35<00:36,  6.81it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  92%|█████████▏| 2997/3246 [07:35<00:36,  6.87it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  92%|█████████▏| 2998/3246 [07:35<00:35,  6.89it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  92%|█████████▏| 2999/3246 [07:35<00:35,  6.89it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  92%|█████████▏| 3000/3246 [07:36<01:20,  3.07it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  92%|█████████▏| 3001/3246 [07:36<01:05,  3.73it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  92%|█████████▏| 3002/3246 [07:37<01:40,  2.43it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  93%|█████████▎| 3003/3246 [07:37<01:20,  3.01it/s, loss=0.197]


Epoch [1/2], Batch [

Epoch [1/2], Batch [1623/1623]:  95%|█████████▍| 3070/3246 [07:57<00:24,  7.20it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  95%|█████████▍| 3071/3246 [07:58<00:26,  6.73it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  95%|█████████▍| 3072/3246 [07:58<00:26,  6.62it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  95%|█████████▍| 3073/3246 [07:58<00:25,  6.85it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  95%|█████████▍| 3074/3246 [07:58<00:34,  4.94it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  95%|█████████▍| 3075/3246 [07:58<00:32,  5.20it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  95%|█████████▍| 3076/3246 [07:58<00:29,  5.72it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  95%|█████████▍| 3077/3246 [07:59<00:27,  6.16it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  95%|█████████▍| 3078/3246 [07:59<00:25,  6.48it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  95%|█████████▍| 3079/3246 [07:59<00:24,  6.73it/s, loss=0.197]


Epoch [1/2], Batch [

Epoch [1/2], Batch [1623/1623]:  97%|█████████▋| 3146/3246 [08:16<00:24,  4.09it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  97%|█████████▋| 3147/3246 [08:16<00:20,  4.74it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  97%|█████████▋| 3148/3246 [08:16<00:18,  5.29it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  97%|█████████▋| 3149/3246 [08:16<00:16,  5.76it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  97%|█████████▋| 3150/3246 [08:17<00:15,  6.11it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  97%|█████████▋| 3151/3246 [08:17<00:14,  6.36it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  97%|█████████▋| 3152/3246 [08:17<00:14,  6.52it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  97%|█████████▋| 3153/3246 [08:17<00:14,  6.64it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  97%|█████████▋| 3154/3246 [08:18<00:27,  3.32it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  97%|█████████▋| 3155/3246 [08:18<00:23,  3.90it/s, loss=0.197]


Epoch [1/2], Batch [

Epoch [1/2], Batch [1623/1623]:  99%|█████████▉| 3222/3246 [08:33<00:03,  6.75it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  99%|█████████▉| 3223/3246 [08:33<00:03,  6.95it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  99%|█████████▉| 3224/3246 [08:33<00:03,  7.09it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  99%|█████████▉| 3225/3246 [08:33<00:02,  7.20it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  99%|█████████▉| 3226/3246 [08:33<00:02,  7.28it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  99%|█████████▉| 3227/3246 [08:33<00:02,  7.33it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  99%|█████████▉| 3228/3246 [08:33<00:02,  7.38it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]:  99%|█████████▉| 3229/3246 [08:34<00:02,  7.42it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]: 100%|█████████▉| 3230/3246 [08:34<00:02,  7.43it/s, loss=0.197]


Epoch [1/2], Batch [1623/1623]: 100%|█████████▉| 3231/3246 [08:34<00:02,  7.43it/s, loss=0.197]


Epoch [1/2], Batch [

Checkpoint saved: ../models/waveform_model_checkpoint_epoch_1.pth
Final model saved to ../models/waveform_model_final.pth


In [69]:
torch.cuda.empty_cache()